# Beating Text-Graph-17 with only Text

Current plan is following

- preprocess text into **q-a connection prediction** (question + question entities [SEP] answer + answer entities (+ linear. graph))
- finetune bert-like model (bigger=better) with some cool LoRA (this one needs to be tuned too)
- abuse augmentations for upsampling minor "correct" label examples

In [1]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch
import os
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
import torch.optim as optim
from sklearn.metrics import precision_score, f1_score, recall_score

In [2]:
SEED = 42

torch.manual_seed(SEED)
torch.random.manual_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.cuda.random.manual_seed(SEED)
torch.cuda.random.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

## Data Preproc

In [3]:
train_path = 'data/tsv/train.tsv'
test_path = 'data/tsv/train.csv'

class TextGraphDataset(Dataset):
    def __init__(self,  tokenizer, max_length, split='train',include_graph=False):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.split = split
        self.include_graph = include_graph
        if split in ['train', 'val', 'test']:
            df = pd.read_csv(train_path, sep='\t')
            df["label"] = df["correct"].astype(np.float32)
            self.df = self._split_train_dev_test(df, split)
        elif split == 'eval': # this corresponds to submit
            self.df = pd.read_csv(test_path, sep='\t')
        else:
            raise ValueError("Unrecognized split!")
        
        self.quetions = []
        self.q_entities = []
        self.a_entities = []
        self.graphs = []
        self.labels = []

        self._get_data()

    def __getitem__(self, idx):
        q_entities = ', '.join(self.q_entities[idx]) + ':'
        question = self.quetions[idx]
        a_entities = ', '.join(self.a_entities[idx])
        
        if self.include_graph:
            raise NotImplementedError("Need to append graph in text form to answer")

        tokenizer_out = self.tokenizer.encode_plus(
            text=q_entities + ' ' + question,
            text_pair=a_entities,
            max_length=self.max_length,
            padding="max_length",
            truncation="only_first",
            return_tensors="pt"
        )

        res = {
            "input_ids": tokenizer_out["input_ids"][0],
            "attention_mask": tokenizer_out["attention_mask"][0],
            "labels": self.labels[idx]
        }
        if "token_type_ids" in tokenizer_out:
            res["token_type_ids"] = tokenizer_out["token_type_ids"][0]
        return res

    def __len__(self):
        return len(self.df)
    
    def _get_data(self):
        for idx, data in self.df.iterrows():
            self.quetions.append(data["question"])
            self.q_entities.append(data["answerEntity"])
            self.a_entities.append(data["questionEntity"])
            self.labels.append(data["label"])
            if self.include_graph:
                self.graphs.append(data["graph"].apply(eval))

    def _split_train_dev_test(self, df, split='train'):
        all_questions = list(df["question"].unique())
        num_questions = len(all_questions)
        random.shuffle(all_questions)

        train_dev_ratio = 0.8
        train_ratio = 0.9
        num_train_dev_questions = int(num_questions * train_dev_ratio)
        train_dev_questions = all_questions[:num_train_dev_questions]
        test_questions = set(all_questions[num_train_dev_questions:])
        num_train_questions = int(len(train_dev_questions) * train_ratio)
        train_questions = set(train_dev_questions[:num_train_questions])
        dev_questions = set(train_dev_questions[num_train_questions:])

        train_df = df[df["question"].isin(train_questions)]
        dev_df = df[df["question"].isin(dev_questions)]
        test_df = df[df["question"].isin(test_questions)]

        if split == 'train':
            return train_df
        elif split =='dev_df':
            return dev_df
        else:
            return test_df

## Model prep and finetuning

In [4]:
# Load model directly
model_name = "whaleloops/phrase-bert"

tokenizer = AutoTokenizer.from_pretrained(model_name)
pretrained_bert = AutoModel.from_pretrained(model_name)

In [5]:
class QuestionClassifier(nn.Module):
    def __init__(self, pretrained_bert):
        super().__init__()
        self.bert_backbone = pretrained_bert
        self.hidden_size = pretrained_bert.config.hidden_size
        self.head = nn.Linear(self.hidden_size, 1)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        outputs = self.bert_backbone(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        last_hidden_state = outputs.last_hidden_state  # Access the last hidden states
        pooled_output = last_hidden_state[:, 0, :]  # Take the [CLS] token representation
        logits = self.head(pooled_output)
        return logits
    
model = QuestionClassifier(
    pretrained_bert
).to(DEVICE)

# freeze backbone
for param in model.bert_backbone.parameters():
    param.requires_grad = False

In [6]:
from peft import LoraConfig, LoraModel

LORA_RANK=4
LORA_ALPHA=100.
LORA_DROPOUT=1e-2

config = LoraConfig(
    task_type="SEQ_CLS",
    r=LORA_RANK,
    lora_alpha=LORA_ALPHA,
    target_modules=["query", "value"],
    lora_dropout=LORA_DROPOUT
)

lora_model = LoraModel(model, config, "default")

In [7]:
def get_trainable_lora_params(model: nn.Module):
    params = []
    for name, p in model.named_parameters():
        if 'lora' in name:
            params.append(p)
    return params

lora_parameters = get_trainable_lora_params(model)
len(lora_parameters)

48

In [8]:
def train_epoch(model, loader, optimizer, loss_fn):
    model.train()

    avg_loss = 0.
    avg_f1 = 0.
    avg_precision = 0.
    avg_recall = 0.
    
    for i, batch in enumerate(loader):

        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(DEVICE)
        token_type_ids = batch["token_type_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        logits = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids).squeeze(1)
        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()

        avg_loss += loss.item()
        with torch.no_grad():
            preds = F.sigmoid(logits).detach().cpu().numpy()
            preds = (preds > 0.5) * 1
            y_true = labels.detach().cpu().numpy()
            avg_f1 += f1_score(y_true, preds)
            avg_precision += precision_score(y_true, preds)
            avg_recall += recall_score(y_true, preds)

        print(y_true)
        print(preds)
        break

    avg_loss /= len(loader) + 1
    avg_f1 /= len(loader) + 1
    avg_precision /= len(loader) + 1
    avg_recall /= len(loader) + 1
    
    return avg_loss, avg_f1, avg_precision, avg_recall


@torch.no_grad
def eval_epoch(model, loader, loss_fn):
    model.eval()

    avg_loss = 0.
    avg_f1 = 0.
    avg_precision = 0.
    avg_recall = 0.

    for i, batch in enumerate(loader):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        token_type_ids = batch["token_type_ids"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        
        logits = model(inputs=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids).squeeze(1)
        loss = loss_fn(logits, labels)

        avg_loss += loss.item()
        preds = F.sigmoid(logits).detach().cpu().numpy()
        preds = (preds > 0.5) * 1
        y_true = labels.detach().cpu().numpy()
        avg_f1 += f1_score(y_true, preds)
        avg_precision += precision_score(y_true, preds)
        avg_recall += recall_score(y_true, preds)

        avg_loss /= len(loader) + 1
        avg_f1 /= len(loader) + 1
        avg_precision /= len(loader) + 1
        avg_recall /= len(loader) + 1

    return avg_loss, avg_f1, avg_precision, avg_recall


def train(model, train_loader, val_loader, optimizer, loss_fn, epochs=10):
    for e in range(epochs):
        loss, f1, prec, recall = train_epoch(model, train_loader, optimizer, loss_fn)
        print(f"Train epoch {e + 1} - loss: {loss:.3f}, f1: {f1:.2f}, precision: {prec:.2f}, recall: {recall:.2f}")

        loss, f1, prec, recall = eval_epoch(model, val_loader, loss_fn)
        print(f"Eval epoch {e + 1} - loss: {loss:.3f}, f1: {f1:.2f}, precision: {prec:.2f}, recall: {recall:.2f}")
        

## Training, evaluation and submit

In [9]:
BATCH_SIZE=64
MAX_LENGTH=128
EPOCHS=50

train_ds = TextGraphDataset(tokenizer, MAX_LENGTH, split='train')
dev_ds = TextGraphDataset(tokenizer, MAX_LENGTH, split='dev')
test_ds = TextGraphDataset(tokenizer, MAX_LENGTH, split='test')

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
dev_loader = DataLoader(dev_ds,batch_size=BATCH_SIZE, shuffle=False, drop_last=True)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

In [10]:
loss_fn = torch.nn.functional.binary_cross_entropy_with_logits
optimizer = optim.Adam(params=list(model.head.parameters())+lora_parameters, lr=1e-3)

In [11]:
import gc
torch.cuda.empty_cache()
gc.collect()

40

In [ ]:
train(
    model,
    train_loader,
    dev_loader,
    optimizer,
    loss_fn,
    epochs=EPOCHS
)

In [ ]:
_, f1, prec, recall = eval_epoch(model, test_loader, loss_fn)
print("Performance on hold-out test - f1 - {f1:.2f}, precision - {prec:.2f}, recall - {recall:.2f}")

In [ ]:
@torch.no_grad
def make_submit_predictions(model, tokenizer, filename='test_result.tsv'):
    model.eval()
    eval_ds = TextGraphDataset(tokenizer, max_length=MAX_LENGTH, split='eval')
    preds = []
    for idx, data in enumerate(eval_ds):
        input_ids = data["input_ids"].to(DEVICE).unsqueeze(0)
        attention_mask = data["attention_mask"].to(DEVICE).unsqueeze(0)
        token_type_ids = data["token_type_ids"].to(DEVICE).unsqueeze(0)
        
        logit = model(inputs=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids).squeeze(1)
        pred = (logit.detach().cpu().numpy() > 0) * 1
        preds.append(pred)

    df = eval_ds.df
    df['prediction'] = preds
    df['prediction'] = df['prediction'].astype(int)
    df[["sample_id", "prediction"]].to_csv(filename, sep='\t', index=False)

In [ ]:
make_submit_predictions(
    model,
    tokenizer
)